In [1]:
import sys
sys.path.append("/auto/homes/fav25/jax/")

import jax.numpy as np
# import autograd.numpy
import numpy as np

from SC_IPFP.sde_solvers import solve_sde_RK
from SC_IPFP.utils import log_kde_pdf_per_point, silvermans_rule

from jax.config import config
from jax import jit, grad, random
from jax.experimental import optimizers
from jax.experimental import stax
from jax.experimental.stax import Dense, Relu, LogSoftmax
import numpy.random as npr
import jax

import itertools
# np = jax.numpy
# np = autograd.numpy

/auto/homes/fav25/jax/jax/lib/xla_bridge.py:116: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
class cIPFP(object):
    
    def __init__(self, X_0, X_1, weights=[100], batch_size=None,  rng = jax.random.PRNGKey(0), 
                nrng = npr.RandomState(0), number_time_steps=16, sde_solver=solve_sde_RK, sigma_sq=1, 
                step_size = 0.001, num_epochs = 10, momentum_mass = 0.9, create_network=None):
        
        self.sde_solver = sde_solver
        
        self.number_time_steps = number_time_steps
        self.dt = 1.0 / number_time_steps
        
        
        self.batch_size_f = X_0.shape[0] if batch_size is None else batch_size
        self.batch_size_b = X_1.shape[0] if batch_size is None else batch_size
        
        
        self.X_0 = X_0
        self.X_1 = X_1
        
        self.H_0 = silvermans_rule(X_0)
        self.H_1 = silvermans_rule(X_1)
        
        _, self.dim = self.X_0.shape
        
        create_net = self.create_network if  create_network is None else create_network
        
        self.b_forward_init, self.b_forward = create_net(
            self.dim, weights
        )
        self.b_backward_init, self.b_backward = create_net(
            self.dim, weights
        )
        
        self.sigma = lambda X,t: sigma_sq
        
        self.rng = rng
        self.nrng = nrng
        
        self.opt_init_f, self.opt_update_f, self.get_params_f = (
            optimizers.momentum(step_size, mass=momentum_mass)
        )
        
        self.opt_init_b, self.opt_update_b, self.get_params_b = (
            optimizers.momentum(step_size, mass=momentum_mass)
        )
        
        num_complete_batches_f, leftover_f = divmod(self.X_0.shape[0], self.batch_size_f)
        self.num_batches_f = num_complete_batches_f + bool(leftover_f)
                                               
        num_complete_batches_b, leftover_b = divmod(self.X_1.shape[0], self.batch_size_b)
        self.num_batches_b = num_complete_batches_b + bool(leftover_b)
        
    
    @staticmethod
    def create_network(dim, weights):
        
        model  = []
        for weight in weights:
            model.append(
                Dense(weight)
            )
                        
            model.append(
                Relu
            )
            
        
        model.append(Dense(dim))
        print(dim)
    
        init_random_params, predict = stax.serial(
           *model
        )
        return init_random_params, predict
        
    @staticmethod
    def loss_for_trajectory(Xt, b_f, b_b, dt, theta, forwards = True):
        b_minus  = b_b(theta, Xt)
        b_plus = b_f(theta, Xt)
        
        delta_Xt = Xt[:-1, :]  - Xt[1:, :]
        
        sign = 1.0 if forwards else -1.0
        
        ito_integral = sign *  (b_plus[1:,:] - b_minus[:-1,:])  * delta_Xt
        
        time_integral = sign *  (b_plus**2 - b_minus**2) * dt # Not sure about this dt here
        
        return ito_integral.sum() - 0.5 * time_integral.sum()
        
    def data_stream(self, forward=True):
        rng = self.nrng
        X = self.X_0 if forward else self.X_1
        
        batch_size = self.batch_size_f if forward else self.batch_size_b
        num_batches = self.num_batches_f if forward else self.num_batches_b
        
        num_train = self.X_0.shape[0] if forward else self.X_1.shape[0]
        while True:
            perm = rng.permutation(num_train)
            for i in range(num_batches):
                batch_idx = perm[i * batch_size:(i + 1) * batch_size]
                yield X[batch_idx] 

#     @jit
    def update(self, i, opt_state, batch, forwards=True):
#         import pdb; pdb.set_trace()

        get_params = self.get_params_f if forwards else self.get_params_b
        params = get_params(opt_state)
        
        loss = lambda params, batch: np.squeeze(self.inner_loss(params, batch, forwards=forwards))
#         import pdb; pdb.set_trace()
        gradient = grad(loss)(params, batch)
    
        opt_update  = self.opt_update_f if forwards else self.opt_update_b
        return opt_update(i, gradient, opt_state)
        
    def sample_trajectory(self, X, theta, forwards=True):
        
        # backwards discretisation has a sign flip         
        b = self.b_forward if forwards else (lambda X, theta: -self.b_backward(X, theta))
        
        return self.sde_solver(alfa=b, beta=self.sigma,
                               dt=self.dt, X0=X,
                               N=self.number_time_steps, theta=theta)
    
    def inner_loss(self, theta, batch, forwards=True):
                       
        J = 0
        terminal_index = -1 if forwards else 0
#         X_terminal_empirical = self.X_1 if forwards else self.X_0
        X_terminal_empirical = next(self.data_stream(forward=not(forwards)))
    
        H = self.H_1 if forwards else self.H_0
        
        for x in batch:
            t, Xt = self.sample_trajectory(x, theta, forwards=forwards)
            
            cross_entropy = log_kde_pdf_per_point(Xt[terminal_index].reshape(-1,1), X_terminal_empirical, H)
            
            J += self.loss_for_trajectory(Xt, self.b_forward, self.b_backward, self.dt, theta, forwards=forwards)
            
            J += cross_entropy
        
        J /= len(batch)
        
        return J
               
    def fit(self, IPFP_iterations=10, sub_iterations=10):     
        
        _, init_params_f = self.b_forward_init(self.rng, (-1, self.dim))                                             
        opt_state_f = self.opt_init_f(init_params_f)
        
        _, init_params_b = self.b_backward_init(self.rng, (-1, self.dim))                                               
        opt_state_b = self.opt_init_b(init_params_b)
        
        batches_f = self.data_stream(forward=True)
        batches_b = self.data_stream(forward=False)
        
        for i in range(IPFP_iterations):
                                               
            itercount = itertools.count()
            
            for k in range(sub_iterations):
                for _ in range(self.num_batches_b):
#                     print(next(itercount), opt_state_b, next(batches_b))
                    opt_state_b = self.update(
                        next(itercount), opt_state_b, next(batches_b), forwards=False
                    )
                                               
            
            itercount = itertools.count()
            
            for k in range(sub_iterations):
                for _ in range(self.num_batches_f):
                    
                    opt_state_f = self.update(
                        next(itercount), opt_state_f, next(batches_f), forwards=True
                    )
         

        self.theta_f = self.get_params_f(opt_state_f)
        self.theta_b = self.get_params_b(opt_state_b)
            
        

In [3]:
X1 = npr.randn(100).reshape(-1,1) - 5
X2 = npr.randn(100).reshape(-1,1) + 5

In [4]:
c = cIPFP(X1, X2,  number_time_steps=100)

1
1


In [5]:
 _, init_params_f = c.b_forward_init(c.rng, (-1, c.dim))                                             
opt_state_f = c.opt_init_f(init_params_f)


theta_f = c.get_params_f(opt_state_f)


In [6]:
theta_f[0][1].shape, theta_f[1]

((100,), ())

In [7]:
t, Xt = c.sample_trajectory(X1[0], theta_f)

TypeError: Argument '<function serial.<locals>.apply_fun at 0x7fcd0409b400>' of type <class 'function'> is not a valid JAX type

In [ ]:
import matplotlib.pyplot as plt

plt.plot(t, Xt.flatten())

In [ ]:
c.dt

In [ ]:
loss= c.loss_for_trajectory(Xt, c.b_forward, c.b_backward, c.dt, theta_f, forwards=True)

In [ ]:
loss

In [ ]:
cross_entropy = log_kde_pdf_per_point(Xt[-1].reshape(-1,1), X2, c.H_0)

In [ ]:
cross_entropy

In [ ]:
c.inner_loss(theta_f, X1)

In [ ]:
c.fit(1,1)